# Imports

In [1]:

from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split

# Reading MIDI files


In [3]:
def read_files(file):
    notes=[]
    notes_to_parse=None
    #parse the midi file
    midi=converter.parse(file)
    #seperate all instruments from the file
    instrmt=instrument.partitionByInstrument(midi)

    for part in instrmt.parts:
        #fetch data only of Piano instrument
        if 'Piano' in str(part):
            # Indented this line to be inside the if block
            notes_to_parse=part.recurse()

        #iterate over all the parts of sub stream elements
        #check if element's type is Note or chord
        #if it is chord split them into notes
        for element in notes_to_parse:
            if type(element)==note.Note:
                notes.append(str(element.pitch))
            elif type(element)==chord.Chord:
                notes.append('.'.join(str(n) for n in element.normalOrder))

    #return the list of notes
    return notes

#retrieve paths recursively from inside the directories/files
file_path=["schubert"]
all_files=glob.glob('All Midi Files/'+file_path[0]+'/*.mid',recursive=True)

#reading each midi file
notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)])

0it [00:00, ?it/s]


# Reading notes

In [5]:
#unique notes
notess = sum(notes_array,[])
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))

#get the threshold frequency
for i in range(30,100,20):
    print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items())))) # Indented this line to be inside the for loop

Unique Notes: 0
30 : 0
50 : 0
70 : 0
90 : 0


In [6]:
#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

In [7]:
#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

In [8]:
#timestep
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
 for j in range(0,len(i)-timesteps):
  #input will be the current index + timestep
  #output will be the next index after timestep
  inp=i[j:j+timesteps] ; out=i[j+timesteps]

  #append the index value of respective notes
  x.append(list(map(lambda x:note2ind[x],inp)))
  y.append(note2ind[out])

x_new=np.array(x)
y_new=np.array(y)